In [ ]:
import requests
import asyncio
import nest_asyncio
import aiohttp
import json
import pandas as pd
import time
from datetime import datetime

In [ ]:
ODDS_API_KEY = "c0945f4cd552fb2d8014c37d90b5463d"
ODDS_API_URL = "https://api.the-odds-api.com/v4/sports/"

BOOKMAKERS = [
    "sport888",  # 888sport
    "betfair_ex_uk",  # Betfair Exchange (UK)
    "betfair_sb_uk",  # Betfair Sportsbook (UK)
    "betvictor",  # Bet Victor
    "betway",  # Betway
    "boylesports",  # BoyleSports
    "casumo",  # Casumo
    "coral",  # Coral
    "grosvenor",  # Grosvenor
    "ladbrokes_uk",  # Ladbrokes (UK)
    "leovegas",  # LeoVegas
    "livescorebet",  # LiveScore Bet
    "matchbook",  # Matchbook
    "paddypower",  # Paddy Power
    "skybet",  # Sky Bet
    "smarkets",  # Smarkets
    "unibet_uk",  # Unibet (UK)
    "virginbet",  # Virgin Bet
    "williamhill",  # William Hill (UK)
    
    # EU Bookies
    "onexbet",  
    "betclic",  
    "betanysports",  
    "betonlineag",  
    "betsson", 
    "coolbet",  
    "everygame",  
    "gtbets",  
    "marathonbet",  
    "mybookieag", 
    "nordicbet",  
    "pinnacle",  
    "suprabets",  
    "tipico_de",  
    "unibet_eu",  
    "winamax_de",  
    "winamax_fr",
]

#  Betting Market 
MARKET = "totals" 

# tgram bot
TELEGRAM_BOT_TOKEN = "7798074179:AAEb3X4phhQFEz-USGFdHEkNipr4hMMnF_M"
TELEGRAM_CHAT_ID = "6179130971"


# Football Leagues to Monitor
FOOTBALL_LEAGUES = [
    "soccer_epl",  # English Premier League
    "soccer_uefa_champs_league",  # UEFA Champions League
    "soccer_italy_serie_a",  # Serie A
    "soccer_spain_la_liga",  # La Liga
    "soccer_germany_bundesliga",  # Bundesliga
    "soccer_france_ligue_one",  # Ligue 1
    "soccer_international_friendly",  # International Friendlies
    "soccer_fifa_world_cup",  # FIFA World Cup
    "soccer_fifa_world_cup_womens",  # FIFA Women's World Cup 
    "soccer_mls",  # Major League Soccer (USA)
    "soccer_uefa_europa_league",  # UEFA Europa League
    "soccer_uefa_europa_conference_league",  # UEFA Europa Conference League
    "soccer_uefa_european_championship",  # UEFA Euro 2024
    "soccer_uefa_euro_qualification",  # UEFA Euro Qualification
    "soccer_uefa_nations_league",  # UEFA Nations League
    "soccer_conmebol_copa_america",  # Copa América
    "soccer_conmebol_copa_libertadores",  # Copa Libertadores
    "soccer_conmebol_copa_sudamericana",  # Copa Sudamericana
    "soccer_argentina_primera_division",  # Argentina Primera Division
    "soccer_australia_aleague",  # Australian A-League
    "soccer_austria_bundesliga",  # Austrian Bundesliga
    "soccer_belgium_first_div",  # Belgian First Division
    "soccer_brazil_campeonato",  # Brazil Série A
    "soccer_brazil_serie_b",  # Brazil Série B
    "soccer_chile_campeonato",  # Chile Primera División
    "soccer_china_superleague",  # Chinese Super League
    "soccer_denmark_superliga",  # Denmark Superliga
    "soccer_efl_champ",  # English Championship
    "soccer_england_efl_cup",  # English EFL Cup
    "soccer_england_league1",  # English League 1
    "soccer_england_league2",  # English League 2
    "soccer_fa_cup",  # English FA Cup
    "soccer_finland_veikkausliiga",  # Finland Veikkausliiga
    "soccer_germany_bundesliga2",  # Bundesliga 2 (Germany)
    "soccer_germany_liga3",  # 3. Liga (Germany)
    "soccer_greece_super_league",  # Greece Super League
    "soccer_japan_j_league",  # Japan J-League
    "soccer_korea_kleague1",  # South Korea K-League 1
    "soccer_league_of_ireland",  # League of Ireland
    "soccer_mexico_ligamx",  # Mexican Liga MX
    "soccer_netherlands_eredivisie",  # Dutch Eredivisie
    "soccer_norway_eliteserien",  # Norway Eliteserien
    "soccer_poland_ekstraklasa",  # Poland Ekstraklasa
    "soccer_portugal_primeira_liga",  # Portugal Primeira Liga
    "soccer_spain_segunda_division",  # Spain Segunda Division
    "soccer_spl",  # Scottish Premiership
    "soccer_sweden_allsvenskan",  # Sweden Allsvenskan
    "soccer_sweden_superettan",  # Sweden Superettan
    "soccer_switzerland_superleague",  # Swiss Super League
    "soccer_turkey_super_league",  # Turkey Super League
]

def fetch_odds(league):
    ODDS_URL = f"https://api.the-odds-api.com/v4/sports/{league}/odds"

    params = {
        "apiKey": ODDS_API_KEY,
        "regions": "uk,eu",  # Ensures odds from UK, US, and Europe
        "markets": MARKET,  # Only Over/Under (totals)
        "oddsFormat": "decimal",
        "bookmakers": ",".join(BOOKMAKERS)
    }

    response = requests.get(ODDS_URL, params=params)

    if response.status_code == 200:
        data = response.json()
        
        available_bookmakers = set()
        for event in data:
            for bookmaker in event.get("bookmakers", []):
                available_bookmakers.add(bookmaker["key"])

        print(f"\n Available bookmakers in {league}: {available_bookmakers}")
        return data
    else:
        print(f" Error fetching odds for {league}: {response.status_code}")
        return None

# Detect Arbitrage Opportunities
def find_arbitrage(odds_data, league):
    arbitrage_opportunities = []

    if not odds_data:
        print(f" No odds data received for {league}.")
        return arbitrage_opportunities

    for event in odds_data:
        event_name = f"{event['home_team']} vs {event['away_team']}"
        under_odds = {}
        over_odds = {}

        print(f"\n Checking event: {event_name} ({league})")

        for bookmaker in event.get("bookmakers", []):
            bookie_name = bookmaker["key"]

            for market in bookmaker.get("markets", []):
                if market["key"] == "totals":  # Only Over/Under markets
                    for outcome in market["outcomes"]:
                        goal_line = outcome.get("point", "N/A")  

                        if "Under" in outcome["name"]:
                            under_odds[(bookie_name, goal_line)] = outcome["price"]
                            print(f"    Under {goal_line} @ {outcome['price']} ({bookie_name})")
                        
                        elif "Over" in outcome["name"]:
                            over_odds[(bookie_name, goal_line)] = outcome["price"]
                            print(f"     Over {goal_line} @ {outcome['price']} ({bookie_name})")

        # Compare Over and Under odds across different bookmakers
        for (bookie1, goal_line), under_price in under_odds.items():
            for (bookie2, goal_line2), over_price in over_odds.items():
                if goal_line == goal_line2 and bookie1 != bookie2:  #  Ensure matching goal lines
                    if check_arbitrage(under_price, over_price):
                        print(f"Arbitrage found: {event_name} - Over/Under {goal_line} Goals")
                        arbitrage_opportunities.append({
                            "event": event_name,
                            "league": league,
                            "goal_line": goal_line,
                            "bookie1": bookie1,
                            "bookie2": bookie2,
                            "outcome1": "Under",
                            "odds1": under_price,
                            "outcome2": "Over",
                            "odds2": over_price,
                            "stake_distribution": calculate_stakes(under_price, over_price)
                        })
                    else:
                        print(f" No arbitrage: Under {under_price} ({bookie1}) vs Over {over_price} ({bookie2})")

    return arbitrage_opportunities


# Arb check
def check_arbitrage(odds1, odds2):
    """Determine if an arbitrage opportunity exists."""
    arb_condition = (1 / odds1) + (1 / odds2)
   # print(f"   Arbitrage Formula: (1/{odds1}) + (1/{odds2}) = {round(arb_condition, 4)}")
    return arb_condition < 1

#  Stake calc
def calculate_stakes(odds1, odds2, total_stake=100):
    """Calculate optimal stake allocation for arbitrage."""
    stake1 = (total_stake / odds1) / ((1 / odds1) + (1 / odds2))
    stake2 = (total_stake / odds2) / ((1 / odds1) + (1 / odds2))
    return {"stake1": round(stake1, 2), "stake2": round(stake2, 2)}

#Send Telegram Alert
async def send_telegram_message(message):
    url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
    payload = {"chat_id": TELEGRAM_CHAT_ID, "text": message, "parse_mode": "Markdown"}
    
    async with aiohttp.ClientSession() as session:
        async with session.post(url, json=payload) as response:
            if response.status != 200:
                print(f" Error sending Telegram message: {await response.text()}")

#  Continuous Arbitrage Scanner 
async def main():
    while True:
        for league in FOOTBALL_LEAGUES:
            print(f"\n Fetching totals odds for {league}...")
            odds_data = fetch_odds(league)

            if odds_data:
                print(f" Odds retrieved for {league}")
                opportunities = find_arbitrage(odds_data, league)

                if opportunities:
                    for opp in opportunities:
                        message = (
                            f" *Arbitrage Opportunity Found!*\n\n"
                            f" *League:* {opp['league']}\n"
                            f" *Match:* {opp['event']}\n"
                            f" *Market:* Over/Under {opp['goal_line']} Goals\n"
                            f" *{opp['outcome1']}* at {opp['bookie1']}: {opp['odds1']}\n"
                            f" *{opp['outcome2']}* at {opp['bookie2']}: {opp['odds2']}\n"
                            f" *Stake Distribution:*\n"
                            f"  - {opp['outcome1']}: ${opp['stake_distribution']['stake1']}\n"
                            f"  - {opp['outcome2']}: ${opp['stake_distribution']['stake2']}\n"
                        )
                        await send_telegram_message(message)
                else:
                    print(f" No arbitrage found for {league}")

        print("\n Waiting 30 seconds before next scan...\n")
        time.sleep(180)
        
#  Run the bot
if __name__ == "__main__":
    nest_asyncio.apply()  # Allow nested async loops in Jupyter
    asyncio.get_event_loop().run_until_complete(main())





⚽ Fetching totals odds for soccer_epl...

✅ Available bookmakers in soccer_epl: {'unibet_eu', 'leovegas', 'onexbet', 'casumo', 'nordicbet', 'tipico_de', 'coolbet', 'unibet_uk', 'betonlineag', 'williamhill', 'pinnacle', 'betsson', 'grosvenor', 'mybookieag'}
✅ Odds retrieved for soccer_epl

📌 Checking event: Arsenal vs Fulham (soccer_epl)
    🔺 Over 2.5 @ 1.93 (mybookieag)
    🔻 Under 2.5 @ 1.78 (mybookieag)
    🔺 Over 2.5 @ 2.04 (onexbet)
    🔻 Under 2.5 @ 1.87 (onexbet)
    🔺 Over 2.5 @ 2.04 (nordicbet)
    🔻 Under 2.5 @ 1.82 (nordicbet)
    🔺 Over 2.5 @ 2.04 (betsson)
    🔻 Under 2.5 @ 1.82 (betsson)
    🔺 Over 2.5 @ 1.9 (tipico_de)
    🔻 Under 2.5 @ 1.85 (tipico_de)
    🔺 Over 2.5 @ 1.91 (williamhill)
    🔻 Under 2.5 @ 1.8 (williamhill)
    🔺 Over 2.5 @ 1.97 (unibet_eu)
    🔻 Under 2.5 @ 1.82 (unibet_eu)
    🔺 Over 2.5 @ 1.92 (leovegas)
    🔻 Under 2.5 @ 1.77 (leovegas)
    🔺 Over 2.5 @ 1.97 (unibet_uk)
    🔻 Under 2.5 @ 1.82 (unibet_uk)
    🔺 Over 2.5 @ 1.97 (casumo)
    🔻 Under 2.

KeyboardInterrupt: 

In [ ]:
import requests

ODDS_API_KEY = "11681d19231a437c683062b7d59932f4"
SPORT_KEY = 'soccer_epl'  # Replace with your desired sport key
REGIONS = 'uk'  # Regions can be uk, us, eu, au
MARKETS = 'h2h'  # Markets can be h2h, spreads, totals
ODDS_FORMAT = 'decimal'  # Odds format can be decimal or american

ODDS_URL = f'https://api.the-odds-api.com/v4/sports/{SPORT_KEY}/odds'

params = {
    'apiKey': ODDS_API_KEY,
    'regions': REGIONS,
    'markets': MARKETS,
    'oddsFormat': ODDS_FORMAT
}

response = requests.get(ODDS_URL, params=params)

if response.status_code == 200:
    odds_data = response.json()
    if odds_data:
        for event in odds_data:
            print(f"Match: {event['home_team']} vs {event['away_team']}")
            for bookmaker in event['bookmakers']:
                print(f"  Bookmaker: {bookmaker['title']}")
                for market in bookmaker['markets']:
                    if market['key'] == 'h2h':
                        for outcome in market['outcomes']:
                            print(f"    {outcome['name']}: {outcome['price']}")
    else:
        print("No odds data available.")
else:
    print(f"Failed to retrieve odds: {response.status_code}")

Failed to retrieve odds: 401
